# Run SuperPrompt-v1 AI Model WEB UI

Make your prompts better for AI Art or in general!

Used Model: https://huggingface.co/roborovski/superprompt-v1

Blog Model: https://brianfitzgerald.xyz/prompt-augmentation/

Google Colab Notebook Made by [Nick088](https://linktr.ee/Nick088)

In [ ]:
#@title Install & Load Dependencies, Model

#@markdown If you wanna use CPU (slower, no daily limit): Set the CPU from Edit -> Notebook Settings -> CPU

#@markdown If you wanna use GPU (faster, max 12 free hours daily limit): Set the Video Card from Edit -> Notebook Settings -> T4 GPU OR Any other GPUs based on your Google Colab Subscription

#@markdown Anyways its a very small model, it doesn't matter much if you use cpu or gpu.
!pip install transformers
!pip install einops
!pip install accelerate
!pip install sentencepiece
!pip install gradio
import gradio as gr
import torch
from IPython.display import clear_output
from transformers import T5Tokenizer, T5ForConditionalGeneration

if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("roborovski/superprompt-v1", torch_dtype=torch.float16)

model.to(device)

clear_output()
print(f"Downloaded SuperPrompt-v1")


In [ ]:
#@title Run UI

def answer(input_text, max_new_tokens, repetition_penalty, temperature, top_p, top_k, seed):
  torch.manual_seed(seed)
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
  outputs = model.generate(input_ids, max_new_tokens=max_new_tokens, repetition_penalty=repetition_penalty, do_sample=True, temperature=temperature, top_p=top_p, top_k=top_k)

  # Extract the generated text from the model output
  dirty_text = tokenizer.decode(outputs[0])
  text = dirty_text.replace("<pad>", "").replace("</s>", "")
  return text

# Define the gradio interface
interface = gr.Interface(
    fn=answer,
    inputs=[
        gr.Textbox(lines=2, label="Your Prompt"),
        gr.Slider(value=512, minimum=250, maximum=512, step=1, interactive=True, label="Max New Tokens", info="The maximum numbers of new tokens, controls how long is the output"),
        gr.Slider(value=1.2, minimum=0, maximum=2, step=0.05, interactive=True, label="Repetition Penalty", info="Penalize repeated tokens, making the AI repeat less itself"),
        gr.Slider(value=0.5, minimum=0, maximum=1, step=0.05, interactive=True, label="Temperature", info="Higher values produce more diverse outputs"),
        gr.Slider(value=1, minimum=0, maximum=2, step=0.05, interactive=True, label="Top P", info="Higher values sample more low-probability tokens"),
        gr.Slider(value=1, minimum=1, maximum=100, step=1, interactive=True, label="Top K", info="Higher k means more diverse outputs by considering a range of tokens"),
        gr.Number(value=42, interactive=True, label="Seed", info="A starting point to initiate the generation process"),
    ],
    outputs="text",
    title="SuperPrompt-v1",
)


# Launch the gradio interface
interface.launch(share=True, debug=True)